In [2]:
import mediapipe as mp
import cv2
import numpy as np
import math

# original plank code

In [ ]:
# mp_pose = mp.solutions.pose
# mp_drawing = mp.solutions.drawing_utils
# pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

# def analyze_plank(video_path, output_video_path=None):
#     cap = cv2.VideoCapture(video_path)
#     frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#     frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#     fps = int(cap.get(cv2.CAP_PROP_FPS))

#     if output_video_path:
#         fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#         out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

#     plank_active = False
#     start_time = None
#     duration = 0
#     frames_without_detection = 0
#     good_frames = 0
#     alignment_scores = []
#     cooldown_frames = 0  # allows for brief loss of pose if user moves

#     print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}")

#     while cap.isOpened():
#         success, image = cap.read()
#         if not success:
#             break

#         image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         results = pose.process(image_rgb)
#         annotated_image = image.copy()

#         if results.pose_landmarks:
#             good_frames += 1
#             frames_without_detection = 0
#             cooldown_frames = 0

#             # Only draw body landmarks (not face)
#             body_landmarks = [
#                 mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER,
#                 mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP,
#                 mp_pose.PoseLandmark.LEFT_ANKLE, mp_pose.PoseLandmark.RIGHT_ANKLE
#             ]
#             # Draw only body landmarks and connections
#             mp_drawing.draw_landmarks(
#                 annotated_image,
#                 results.pose_landmarks,
#                 mp_pose.POSE_CONNECTIONS,
#                 mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
#                 mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
#             )

#             landmarks = results.pose_landmarks.landmark

#             left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
#                              landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
#             right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
#                               landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
#             left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
#                         landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
#             right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
#                          landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
#             left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
#                           landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
#             right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
#                            landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

#             def calculate_angle(a, b, c):
#                 a = np.array(a)
#                 b = np.array(b)
#                 c = np.array(c)
#                 radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
#                 angle = np.abs(radians * 180.0 / np.pi)
#                 if angle > 180.0:
#                     angle = 360 - angle
#                 return angle

#             # Check body alignment (shoulder-hip-ankle angle should be close to 180)
#             left_alignment_angle = calculate_angle(left_shoulder, left_hip, left_ankle)
#             right_alignment_angle = calculate_angle(right_shoulder, right_hip, right_ankle)
#             alignment_score = (180 - abs(left_alignment_angle - 180) + 180 - abs(right_alignment_angle - 180)) / 360
#             alignment_scores.append(alignment_score)

#             cv2.putText(annotated_image, f"Alignment: {alignment_score * 100:.1f}%", (10, 30),
#                         cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)

#             # Determine if plank is active (shoulders above hips, ignore feet)
#             if left_shoulder[1] < left_hip[1] and right_shoulder[1] < right_hip[1]:
#                 if not plank_active:
#                     plank_active = True
#                     start_time = cap.get(cv2.CAP_PROP_POS_MSEC)
#                 if start_time is not None:
#                     current_time = cap.get(cv2.CAP_PROP_POS_MSEC)
#                     duration = (current_time - start_time) / 1000  # seconds
#             else:
#                 # Instead of immediately resetting, allow a short cooldown
#                 if plank_active and cooldown_frames < fps // 2:  # allow 0.5s grace
#                     cooldown_frames += 1
#                 else:
#                     plank_active = False
#                     start_time = None
#                     duration = 0
#         else:
#             frames_without_detection += 1
#             if frames_without_detection % 30 == 0:
#                 print(f"No pose detection for {frames_without_detection} frames")
#             # Allow brief detection loss
#             if plank_active and cooldown_frames < fps // 2:
#                 cooldown_frames += 1
#             else:
#                 plank_active = False
#                 start_time = None
#                 duration = 0

#         cv2.putText(annotated_image, f'Duration: {duration:.1f}s', (10, 70),
#                     cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

#         if output_video_path:
#             out.write(annotated_image)

#     cap.release()
#     if output_video_path:
#         out.release()

#     if good_frames < 10:
#         return {
#             "duration": 0,
#             "error": "Not enough valid pose detections. Check video quality and positioning."
#         }

#     avg_alignment = sum(alignment_scores) / max(len(alignment_scores), 1)

#     feedback = {
#         "duration": duration,
#         "form_analysis": {
#             "avg_body_alignment_score": avg_alignment * 100,
#             "frames_analyzed": good_frames
#         },
#         "feedback": []
#     }

#     if avg_alignment < 0.85:
#         feedback["feedback"].append("Maintain a straighter line from your shoulders to your ankles. Avoid sagging or arching your back.")

#     if not feedback["feedback"]:
#         feedback["feedback"].append("Great plank form! You held a good straight line.")

#     return feedback


I0000 00:00:1746083487.481316 1978783 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3


In [ ]:
#video_file = 'planks/dom_plank.mp4'
# output_file = 'planks/dom_output.mov'


# analysis_results = analyze_plank(video_file, output_video_path=output_file)
# print("\n--- Plank Analysis Results ---")


# print(f"Total Plank Duration: {analysis_results['duration']} seconds")

# print("\nForm Analysis:")
# for key, value in analysis_results['form_analysis'].items():
#     print(f"  {key.replace('_', ' ').title()}: {value}")

# print("\nFeedback:")
# for msg in analysis_results['feedback']:
#     print(f"- {msg}")



# altnerative

In [13]:
import mediapipe as mp
import cv2
import numpy as np
import math

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose_tracker = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def detect_orientation(lm):
    """
    Determine whether subject is facing 'front', 'left', or 'right' based on nose and shoulder position.
    This is adapted from the band pull-apart code.
    """
    # Check if necessary landmarks are available
    if (mp_pose.PoseLandmark.NOSE.value >= len(lm) or
        mp_pose.PoseLandmark.LEFT_SHOULDER.value >= len(lm) or
        mp_pose.PoseLandmark.RIGHT_SHOULDER.value >= len(lm)):
        return "unknown" # Return unknown if landmarks are missing

    nose = lm[mp_pose.PoseLandmark.NOSE.value]
    ls = lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    rs = lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]

    # Check if landmark visibility is high enough
    if nose.visibility < 0.5 or ls.visibility < 0.5 or rs.visibility < 0.5:
         return "unknown" # Return unknown if visibility is low

    mid_x = (ls.x + rs.x) / 2
    delta = nose.x - mid_x

    # Use a threshold to determine orientation
    # These thresholds might need tuning based on typical video perspectives
    if abs(delta) < 0.05: # Subject is likely facing forward
        return "front"
    # Subject is facing right if nose is to the left of the midpoint
    return "right" if delta < 0 else "left" # Subject is facing left if nose is to the right


def calculate_angle(a, b, c):
    """
    Calculates the angle in degrees between three points a, b, and c, with b being the vertex.
    """
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    # Calculate the angle using arctan2
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    # Ensure the angle is between 0 and 180 degrees
    if angle > 180.0:
        angle = 360 - angle
    return angle


def analyze_plank(video_path, output_video_path=None):
    """
    Analyzes a video of a person performing a plank exercise,
    calculating duration, overall body alignment, and providing feedback
    based on detected pose and orientation.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return {"duration": 0, "error": "Could not open video file."}

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    writer = None
    if output_video_path:
        # Define the codec and create VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'mp4v') # You can use 'XVID' or 'MJPG' if 'mp4v' doesn't work
        try:
            writer = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
        except Exception as e:
            print(f"Error creating video writer: {e}")
            writer = None # Ensure writer is None if creation fails


    plank_active = False
    start_time_msec = None # Store start time in milliseconds
    duration = 0 # Duration in seconds
    frames_without_detection = 0
    good_frames = 0 # Frames with successful pose detection
    alignment_scores = [] # List to store alignment scores per frame
    orientations = [] # List to store detected orientations per frame
    cooldown_frames = 0 # Allows for brief loss of pose if user moves

    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}")

    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break # End of video or error reading frame

        # Convert the image to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Process the image to find pose landmarks
        results = pose_tracker.process(image_rgb)
        # Create a copy of the original image to draw on
        annotated_image = image.copy()

        if results.pose_landmarks:
            good_frames += 1
            frames_without_detection = 0 # Reset counter on successful detection
            cooldown_frames = 0 # Reset cooldown on successful detection

            landmarks = results.pose_landmarks.landmark

            # Detect orientation for the current frame
            orientations.append(detect_orientation(landmarks))


            # Define key body landmarks for plank analysis
            try:
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                 landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                  landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                              landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                               landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                # Also get ear and nose for more specific feedback later if needed
                left_ear = [landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y]
                right_ear = [landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].y]
                nose = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,
                        landmarks[mp_pose.PoseLandmark.NOSE.value].y]

            except Exception as e:
                print(f"Error accessing landmarks: {e}")
                # Skip analysis for this frame if landmarks are missing or invalid
                if writer:
                    writer.write(annotated_image) # Still write the frame even without analysis
                continue # Move to the next frame


            # Check body alignment (shoulder-hip-ankle angle should be close to 180 for a straight line)
            # Calculate angles for both sides and average them
            left_alignment_angle = calculate_angle(left_shoulder, left_hip, left_ankle)
            right_alignment_angle = calculate_angle(right_shoulder, right_hip, right_ankle)

            # A simple score: 180 - deviation from 180, normalized. Higher is better.
            # Max possible score for one side is 180 (perfectly straight). Averaging two sides.
            alignment_score = ( (180 - abs(left_alignment_angle - 180)) + (180 - abs(right_alignment_angle - 180)) ) / 360.0
            alignment_scores.append(alignment_score)

            # Draw only body landmarks and connections on the annotated image
            # You can customize the drawing spec colors and thickness
            mp_drawing.draw_landmarks(
                annotated_image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2), # Example color for points
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2) # Example color for connections
            )

            # Overlay alignment score on the video frame
            cv2.putText(annotated_image, f"Alignment: {alignment_score * 100:.1f}%", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)

            # Determine if plank is active
            # Condition: Shoulders are generally above hips vertically.
            # This is a simplified check and might need refinement based on camera angle.
            # We ignore ankle position for activation check as feet can be higher/lower depending on plank type (forearm vs full)
            # and camera angle.
            is_shoulders_above_hips = left_shoulder[1] < left_hip[1] and right_shoulder[1] < right_hip[1]

            if is_shoulders_above_hips:
                if not plank_active:
                    plank_active = True
                    start_time_msec = cap.get(cv2.CAP_PROP_POS_MSEC) # Record start time
                if start_time_msec is not None:
                    current_time_msec = cap.get(cv2.CAP_PROP_POS_MSEC)
                    duration = (current_time_msec - start_time_msec) / 1000.0 # Calculate duration in seconds
            else:
                # If pose is not clearly a plank, start cooldown
                if plank_active and cooldown_frames < fps // 2: # Allow 0.5s grace period
                    cooldown_frames += 1
                else:
                    # If cooldown expires or plank wasn't active, reset
                    plank_active = False
                    start_time_msec = None
                    duration = 0 # Reset duration if plank is no longer active

        else:
            # No pose detection in this frame
            frames_without_detection += 1
            if frames_without_detection % 30 == 0: # Print a message every second of no detection
                print(f"No pose detection for {frames_without_detection} frames")

            # If plank was active, allow a grace period for detection loss
            if plank_active and cooldown_frames < fps // 2:
                cooldown_frames += 1
            else:
                # If cooldown expires or plank wasn't active, reset
                plank_active = False
                start_time_msec = None
                duration = 0


        # Overlay duration on the video frame
        cv2.putText(annotated_image, f'Duration: {duration:.1f}s', (10, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        # Write the annotated frame to the output video
        if writer:
            writer.write(annotated_image)

    # Release video capture and writer objects
    cap.release()
    if writer:
        writer.release()

    # --- Analysis Summary and Feedback ---

    if good_frames < fps * 2: # Require at least 2 seconds of good detection for analysis
        return {
            "duration": duration, # Return the final duration recorded
            "error": "Not enough valid pose detections for detailed analysis. Ensure subject is fully visible and lighting is good."
        }

    # Calculate average alignment score over all good frames
    avg_alignment = sum(alignment_scores) / max(len(alignment_scores), 1)

    # Determine the dominant view during the plank
    # Filter out "unknown" orientations before finding the dominant one
    valid_orientations = [o for o in orientations if o != "unknown"]
    dominant_view = "unknown"
    if valid_orientations:
        dominant_view = max(set(valid_orientations), key=valid_orientations.count)


    # Prepare the feedback dictionary
    feedback_results = {
        "duration": duration,
        "form_analysis": {
            "avg_body_alignment_score": avg_alignment * 100, # Convert to percentage
            "frames_analyzed": good_frames,
            "dominant_view": dominant_view
        },
        "feedback": [] # List to hold specific feedback messages
    }

    # Add general feedback based on average alignment
    if avg_alignment < 0.70: # Lower threshold for poor alignment
         feedback_results["feedback"].append("Significant sagging or arching detected. Focus on maintaining a straight line from shoulders to ankles.")
    elif avg_alignment < 0.85: # Moderate threshold for improvement
        feedback_results["feedback"].append("Maintain a straighter line from your shoulders to your ankles. Avoid slight sagging or arching.")
    else:
         feedback_results["feedback"].append("Good overall body alignment!")


    # Add orientation-specific feedback
    if dominant_view == "front":
        feedback_results["feedback"].append("Front view detected: Ensure shoulders are directly over elbows and hips are level.")
        # Could add checks for hip sway side-to-side if needed
    elif dominant_view == "left":
        feedback_results["feedback"].append("Side view (left) detected: Focus on keeping your left hip from dropping and maintaining a straight line.")
        # Could add checks for hip height relative to shoulder/ankle
    elif dominant_view == "right":
        feedback_results["feedback"].append(f"Side view (right) detected: Focus on keeping your right hip from dropping and maintaining a straight line.")
        # Could add checks for hip height relative to shoulder/ankle
    else: # dominant_view == "unknown"
         feedback_results["feedback"].append("Could not determine clear orientation. For best analysis, ensure your side or front is clearly visible to the camera.")


    # If no specific feedback was added, provide a positive message
    if len(feedback_results["feedback"]) == 3 and "Good overall body alignment!" in feedback_results["feedback"]: # Only general and orientation feedback
         feedback_results["feedback"].append("Excellent plank form!")
    elif not feedback_results["feedback"]: # Should not happen with general feedback, but as a fallback
         feedback_results["feedback"].append("Analysis complete. Reviewing your form.")


    return feedback_results

I0000 00:00:1746431192.197813 2213824 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3


W0000 00:00:1746431192.326460 3988323 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746431192.346102 3988323 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [14]:
# Make sure you have a video file named 'your_plank_video.mp4' in the same directory
# Or provide the full path to your video file
video_file = 'dom_plank.mp4'
output_file = 'dom_output_2.mov' # Output video with annotations
# Call the analysis function
analysis_summary = analyze_plank(video_file, output_file)
# Print the results
print("\n--- Plank Analysis Summary ---")
if "error" in analysis_summary:
    print(f"Error: {analysis_summary['error']}")
else:
    print(f"Plank Duration: {analysis_summary['duration']:.1f} seconds")
    print(f"Average Body Alignment Score: {analysis_summary['form_analysis']['avg_body_alignment_score']:.1f}%")
    print(f"Dominant View: {analysis_summary['form_analysis']['dominant_view']}")
    print("\nFeedback:")
    for item in analysis_summary['feedback']:
        print(f"- {item}")

Video dimensions: 1280x720, FPS: 30

--- Plank Analysis Summary ---
Plank Duration: 61.7 seconds
Average Body Alignment Score: 96.5%
Dominant View: left

Feedback:
- Good overall body alignment!
- Side view (left) detected: Focus on keeping your left hip from dropping and maintaining a straight line.


# the one being used in the script

In [19]:
import mediapipe as mp
import cv2
import numpy as np
import math

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def detect_orientation(lm):
    """
    Determine whether subject is facing 'front', 'left', or 'right'.
    """
    nose = lm[mp_pose.PoseLandmark.NOSE.value]
    ls = lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    rs = lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    mid_x = (ls.x + rs.x) / 2
    delta = nose.x - mid_x
    if abs(delta) < 0.05:
        return "front"
    return "right" if delta < 0 else "left"

def analyze_plank(video_path, output_video_path=None):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    plank_active = False
    start_time = None
    duration = 0
    frames_without_detection = 0
    good_frames = 0
    cooldown_frames = 0  # allows for brief loss of pose if user moves
    
    # Metrics to track
    alignment_scores = []
    hip_drops = []
    shoulder_heights = []
    orientations = []
    back_angles = []
    neck_angles = []

    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}")

    def calculate_angle(a, b, c):
        a = np.array(a)
        b = np.array(b)
        c = np.array(c)
        radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
        angle = np.abs(radians * 180.0 / np.pi)
        if angle > 180.0:
            angle = 360 - angle
        return angle

    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        annotated_image = image.copy()

        if results.pose_landmarks:
            good_frames += 1
            frames_without_detection = 0
            cooldown_frames = 0

            landmarks = results.pose_landmarks.landmark
            orientations.append(detect_orientation(landmarks))

            # Get key points
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            nose = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,
                   landmarks[mp_pose.PoseLandmark.NOSE.value].y]
            left_ear = [landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y]
            right_ear = [landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].y]

            # Calculate metrics
            # 1. Body alignment (shoulder-hip-ankle)
            left_alignment_angle = calculate_angle(left_shoulder, left_hip, left_ankle)
            right_alignment_angle = calculate_angle(right_shoulder, right_hip, right_ankle)
            alignment_score = (180 - abs(left_alignment_angle - 180) + 180 - abs(right_alignment_angle - 180)) / 360
            alignment_scores.append(alignment_score)

            # 2. Hip drop (difference between shoulder and hip height)
            avg_shoulder_y = (left_shoulder[1] + right_shoulder[1]) / 2
            avg_hip_y = (left_hip[1] + right_hip[1]) / 2
            hip_drop = (avg_hip_y - avg_shoulder_y) * frame_height  # in pixels
            hip_drops.append(hip_drop)

            # 3. Shoulder height difference (uneven shoulders)
            shoulder_height_diff = abs(left_shoulder[1] - right_shoulder[1]) * frame_height
            shoulder_heights.append(shoulder_height_diff)

            # 4. Back angle (shoulder-hip-knee, for sagging detection)
            left_back_angle = calculate_angle(left_shoulder, left_hip, left_knee)
            right_back_angle = calculate_angle(right_shoulder, right_hip, right_knee)
            back_angles.append((left_back_angle + right_back_angle) / 2)

            # 5. Neck angle (ear-nose-ear)
            neck_angle = calculate_angle(left_ear, nose, right_ear)
            neck_angles.append(neck_angle)

            # Draw pose landmarks
            mp_drawing.draw_landmarks(
                annotated_image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )

            # Display metrics
            cv2.putText(annotated_image, f"Alignment: {alignment_score * 100:.1f}%", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(annotated_image, f"Hip Drop: {hip_drop:.1f}px", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(annotated_image, f"Back Angle: {back_angles[-1]:.1f}°", (10, 90),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2, cv2.LINE_AA)

            # Determine if plank is active (shoulders above hips)
            if left_shoulder[1] < left_hip[1] and right_shoulder[1] < right_hip[1]:
                if not plank_active:
                    plank_active = True
                    start_time = cap.get(cv2.CAP_PROP_POS_MSEC)
                if start_time is not None:
                    current_time = cap.get(cv2.CAP_PROP_POS_MSEC)
                    duration = (current_time - start_time) / 1000  # seconds
            else:
                if plank_active and cooldown_frames < fps // 2:  # allow 0.5s grace
                    cooldown_frames += 1
                else:
                    plank_active = False
                    start_time = None
                    duration = 0
        else:
            frames_without_detection += 1
            if frames_without_detection % 30 == 0:
                print(f"No pose detection for {frames_without_detection} frames")
            if plank_active and cooldown_frames < fps // 2:
                cooldown_frames += 1
            else:
                plank_active = False
                start_time = None
                duration = 0

        cv2.putText(annotated_image, f'Duration: {duration:.1f}s', (10, frame_height - 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        if output_video_path:
            out.write(annotated_image)

    cap.release()
    if output_video_path:
        out.release()

    if good_frames < 10:
        return {
            "duration": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }

    # Calculate averages
    avg_alignment = sum(alignment_scores) / len(alignment_scores)
    avg_hip_drop = sum(hip_drops) / len(hip_drops)
    avg_shoulder_diff = sum(shoulder_heights) / len(shoulder_heights)
    avg_back_angle = sum(back_angles) / len(back_angles)
    avg_neck_angle = sum(neck_angles) / len(neck_angles)
    dominant_view = max(set(orientations), key=orientations.count) if orientations else "unknown"

    # Prepare feedback
    feedback = {
        "duration": duration,
        "form_analysis": {
            "avg_body_alignment_score": avg_alignment * 100,
            "avg_hip_drop_px": avg_hip_drop,
            "avg_shoulder_height_diff_px": avg_shoulder_diff,
            "avg_back_angle": avg_back_angle,
            "avg_neck_angle": avg_neck_angle,
            "dominant_view": dominant_view,
            "frames_analyzed": good_frames
        },
        "feedback": []
    }

    # General feedback
    if avg_alignment < 0.85:
        feedback["feedback"].append("Maintain a straighter line from shoulders to ankles. Avoid sagging or arching your back.")
    if avg_hip_drop > 0.1 * frame_height:  # More than 10% of frame height
        feedback["feedback"].append("Engage your core to prevent your hips from sagging.")
    if avg_shoulder_diff > 0.05 * frame_height:
        feedback["feedback"].append("Keep your shoulders level - one side appears higher than the other.")
    if avg_back_angle < 160:
        feedback["feedback"].append("Keep your back straighter - you may be rounding your shoulders or arching your lower back.")
    if avg_neck_angle < 160:
        feedback["feedback"].append("Keep your neck in line with your spine - avoid looking forward, dropping your head, or any movements.")

    # View-specific feedback
    if dominant_view == "front":
        feedback["feedback"].append("Front view: Ensure your hands are shoulder-width apart and body is straight.")
    elif dominant_view == "left":
        feedback["feedback"].append("Left side view: Check that your hips aren't sagging and your shoulders are directly above your elbows.")
    elif dominant_view == "right":
        feedback["feedback"].append("Right side view: Check that your hips aren't sagging and your shoulders are directly above your elbows.")

    return feedback

I0000 00:00:1746431687.016253 2213824 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3


W0000 00:00:1746431687.138608 3992858 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746431687.161028 3992857 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [20]:
# Make sure you have a video file named 'your_plank_video.mp4' in the same directory
# Or provide the full path to your video file
video_file = 'dom_plank.mp4'
output_file = 'dom_output_2.mov' # Output video with annotations
# Call the analysis function
analysis_summary = analyze_plank(video_file, output_file)
# Print the results
print("\n--- Plank Analysis Summary ---")
if "error" in analysis_summary:
    print(f"Error: {analysis_summary['error']}")
else:
    print(f"Plank Duration: {analysis_summary['duration']:.1f} seconds")
    print(f"Average Body Alignment Score: {analysis_summary['form_analysis']['avg_body_alignment_score']:.1f}%")
    print(f"Dominant View: {analysis_summary['form_analysis']['dominant_view']}")
    print("\nFeedback:")
    for item in analysis_summary['feedback']:
        print(f"- {item}")

Video dimensions: 1280x720, FPS: 30

--- Plank Analysis Summary ---
Plank Duration: 61.7 seconds
Average Body Alignment Score: 96.5%
Dominant View: left

Feedback:
- Keep your neck in line with your spine - avoid looking forward, dropping your head, or any movements.
- Left side view: Check that your hips aren't sagging and your shoulders are directly above your elbows.


# another alternative 2

In [21]:
# import mediapipe as mp
# import cv2
# import numpy as np
# import math

# mp_pose = mp.solutions.pose
# mp_drawing = mp.solutions.drawing_utils
# pose_tracker = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

# def detect_orientation(lm):
#     """Determine subject orientation: 'front', 'left', or 'right'."""
#     nose = lm[mp_pose.PoseLandmark.NOSE.value]
#     ls   = lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
#     rs   = lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
#     mid_x = (ls.x + rs.x) / 2
#     delta = nose.x - mid_x
#     if abs(delta) < 0.05:
#         return "front"
#     return "right" if delta < 0 else "left"

# def calc_angle(a, b, c):
#     """Calculate angle (in degrees) at point b given three points."""
#     a, b, c = np.array(a), np.array(b), np.array(c)
#     ang = abs(np.degrees(
#         np.arctan2(c[1]-b[1], c[0]-b[0]) -
#         np.arctan2(a[1]-b[1], a[0]-b[0])
#     ))
#     return 360 - ang if ang > 180 else ang

# def analyze_plank(video_path, output_video_path=None):
#     cap = cv2.VideoCapture(video_path)
#     w, h = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#     fps = cap.get(cv2.CAP_PROP_FPS)
#     writer = None
#     if output_video_path:
#         fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#         writer = cv2.VideoWriter(output_video_path, fourcc, fps, (w, h))

#     print(f"Video dimensions: {w}x{h}, FPS: {fps}")

#     plank_active = False
#     start_time = None
#     duration = 0
#     frames_without_detection = 0
#     good_frames = 0
#     cooldown_frames = 0

#     align_scores = []
#     hip_angles = []
#     neck_angles = []
#     shoulder_heights = []
#     orientations = []

#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         res = pose_tracker.process(img_rgb)
#         out = frame.copy()

#         if res.pose_landmarks:
#             good_frames += 1
#             frames_without_detection = 0
#             cooldown_frames = 0
#             lm = res.pose_landmarks.landmark

#             # Orientation
#             orientation = detect_orientation(lm)
#             orientations.append(orientation)

#             # helpers
#             def nc(idx): return [lm[idx].x, lm[idx].y]
#             def px(idx): return (int(lm[idx].x * w), int(lm[idx].y * h))

#             # key points
#             ls, rs = nc(mp_pose.PoseLandmark.LEFT_SHOULDER.value), nc(mp_pose.PoseLandmark.RIGHT_SHOULDER.value)
#             lh, rh = nc(mp_pose.PoseLandmark.LEFT_HIP.value), nc(mp_pose.PoseLandmark.RIGHT_HIP.value)
#             la, ra = nc(mp_pose.PoseLandmark.LEFT_ANKLE.value), nc(mp_pose.PoseLandmark.RIGHT_ANKLE.value)
#             le, re = nc(mp_pose.PoseLandmark.LEFT_ELBOW.value), nc(mp_pose.PoseLandmark.RIGHT_ELBOW.value)
#             lw, rw = nc(mp_pose.PoseLandmark.LEFT_WRIST.value), nc(mp_pose.PoseLandmark.RIGHT_WRIST.value)
#             nose   = nc(mp_pose.PoseLandmark.NOSE.value)
#             leye   = nc(mp_pose.PoseLandmark.LEFT_EYE.value)
#             reye   = nc(mp_pose.PoseLandmark.RIGHT_EYE.value)

#             # Alignment: shoulder-hip-ankle angle both sides
#             left_align  = calc_angle(ls, lh, la)
#             right_align = calc_angle(rs, rh, ra)
#             align_score = (180 - abs(left_align - 180) + 180 - abs(right_align - 180)) / 360
#             align_scores.append(align_score)

#             # Hip angle: shoulder-hip-ankle
#             avg_hip_angle = (left_align + right_align) / 2
#             hip_angles.append(avg_hip_angle)

#             # Neck angle (shoulder-shoulder-nose)
#             neck_angle = calc_angle(ls, nose, rs)
#             neck_angles.append(neck_angle)

#             # Shoulder height diff (sag or tilt)
#             shoulder_heights.append(abs((ls[1] - rs[1]) * h))

#             # Draw pose
#             mp_drawing.draw_landmarks(out, res.pose_landmarks, mp_pose.POSE_CONNECTIONS)
#             # Overlays
#             cv2.putText(out, f"Align: {align_score*100:.1f}%", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
#             cv2.putText(out, f"Hip: {avg_hip_angle:.1f}°", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
#             cv2.putText(out, f"Neck: {neck_angle:.1f}°", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,0), 2)
#             cv2.putText(out, f"View: {orientation}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,255), 2)

#             # Plank active logic (shoulders above hips)
#             if ls[1] < lh[1] and rs[1] < rh[1]:
#                 if not plank_active:
#                     plank_active = True
#                     start_time = cap.get(cv2.CAP_PROP_POS_MSEC)
#                 if start_time is not None:
#                     current_time = cap.get(cv2.CAP_PROP_POS_MSEC)
#                     duration = (current_time - start_time) / 1000  # seconds
#             else:
#                 if plank_active and cooldown_frames < fps // 2:
#                     cooldown_frames += 1
#                 else:
#                     plank_active = False
#                     start_time = None
#                     duration = 0
#         else:
#             frames_without_detection += 1
#             if frames_without_detection % 30 == 0:
#                 print(f"No pose detection for {frames_without_detection} frames")
#             if plank_active and cooldown_frames < fps // 2:
#                 cooldown_frames += 1
#             else:
#                 plank_active = False
#                 start_time = None
#                 duration = 0

#         cv2.putText(out, f'Duration: {duration:.1f}s', (10, h-30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
#         if writer:
#             writer.write(out)

#     cap.release()
#     if writer:
#         writer.release()

#     if good_frames < 10:
#         return {
#             "duration": 0,
#             "error": "Not enough valid pose detections. Check video quality and positioning."
#         }

#     avg_align = sum(align_scores) / max(len(align_scores), 1)
#     avg_hip = sum(hip_angles) / max(len(hip_angles), 1)
#     avg_neck = sum(neck_angles) / max(len(neck_angles), 1)
#     avg_shoulder_dy = sum(shoulder_heights) / max(len(shoulder_heights), 1)
#     dominant_view = max(set(orientations), key=orientations.count)

#     feedback = []
#     # Alignment feedback
#     if avg_align < 0.85:
#         feedback.append("Keep your body in a straight line from shoulders to ankles-avoid sagging or arching your back.")
#     # Hip angle feedback
#     if avg_hip < 165:
#         feedback.append("Raise your hips slightly to avoid sagging.")
#     elif avg_hip > 195:
#         feedback.append("Lower your hips to avoid piking up.")
#     # Neck feedback
#     if avg_neck < 165:
#         feedback.append("Keep your neck neutral-look straight down, not forward.")
#     # Shoulder feedback
#     if avg_shoulder_dy > 0.05 * h:
#         feedback.append("Keep your shoulders level-avoid tilting or twisting.")
#     # Orientation feedback
#     if dominant_view == "front":
#         feedback.append("Front view: ensure hips and shoulders are level and avoid twisting.")
#     elif dominant_view == "left":
#         feedback.append("Side view (left): check for straight line from head to heels and neutral neck.")
#     else:
#         feedback.append("Side view (right): check for straight line from head to heels and neutral neck.")
#     if not feedback:
#         feedback.append("Excellent plank form! You held a strong, straight line.")

#     return {
#         "duration": duration,
#         "form_analysis": {
#             "avg_body_alignment_score": avg_align * 100,
#             "avg_hip_angle": avg_hip,
#             "avg_neck_angle": avg_neck,
#             "avg_shoulder_height_diff_px": avg_shoulder_dy,
#             "dominant_view": dominant_view,
#             "frames_analyzed": good_frames
#         },
#         "feedback": feedback
#     }


I0000 00:00:1746431800.531777 2213824 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3


W0000 00:00:1746431800.690111 3994212 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746431800.709336 3994214 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [22]:
# result = analyze_plank("dom_plank.mp4", "annotated_plank.mp4")

# print(f"\n=== PLANK ANALYSIS RESULTS ===")
# if "error" in result:
#     print(f"Error: {result['error']}")
# else:
#     print(f"Duration held: {result['duration']:.1f} seconds")
#     print(f"Average alignment: {result['form_analysis']['avg_body_alignment_score']:.1f}%")
#     print(f"Dominant view: {result['form_analysis']['dominant_view']}")
#     print(f"Frames analyzed: {result['form_analysis']['frames_analyzed']}")
#     print("\nFeedback:")
#     for item in result["feedback"]:
#         print(f"- {item}")


Video dimensions: 1280x720, FPS: 30.01234596324414

=== PLANK ANALYSIS RESULTS ===
Duration held: 61.7 seconds
Average alignment: 96.5%
Dominant view: left
Frames analyzed: 1854

Feedback:
- Keep your neck neutral-look straight down, not forward.
- Side view (left): check for straight line from head to heels and neutral neck.
